In [ ]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

<h1> Define properties of optical system </h1>

In [ ]:
# all measurements are in meters

FOCAL_LENGTH = 0.0107
BASELINE = 0.135
PIXEL_SIZE_M = 3.45 * 1e-6
FOCAL_LENGTH_PIXEL = FOCAL_LENGTH / PIXEL_SIZE_M
IMAGE_SENSOR_WIDTH = 0.01412
IMAGE_SENSOR_HEIGHT = 0.01035
PIXEL_COUNT_WIDTH = 4096
PIXEL_COUNT_HEIGHT = 3000




<h1> Perform Sensitivity Analysis </h1>

<h2> Our goal here is to determine how sensitive our distance predictions are to pixel disparity errors. This will depend on how big the fish is, how far the fish is away from the camera, and possibly where the fish is located in the image (i.e. in the middle, near the edge, etc.) </h2>

<h3> Simple case: Take a fish of known length (i.e head to tail distance) at a known distance away from camera, centered in the image. How sensitive is fish length as a function of disparity error? </h3>

In [ ]:
def convert_to_world_point(x, y, d):
    """ from pixel coordinates to world coordinates """
    # TODO (@Thomas) this is hard coded and this bad....
    image_center_x = PIXEL_COUNT_WIDTH / 2.0  # depth_map.shape[1] / 2.0
    image_center_y = PIXEL_COUNT_HEIGHT / 2.0  # depth_map.shape[0] / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (IMAGE_SENSOR_WIDTH / PIXEL_COUNT_WIDTH)
    sensor_z = px_z * (IMAGE_SENSOR_HEIGHT / PIXEL_COUNT_HEIGHT)

    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / FOCAL_LENGTH
    world_z = (world_y * sensor_z) / FOCAL_LENGTH
    return np.array([world_x, world_y, world_z])
    

In [ ]:
def get_length_error(fish_length, distance_from_camera, head_disparity_error_px, tail_disparity_error_px):
    '''
    determine distance error as a function of disparity error (assume diaprity error impacts only the head)
    '''
    
    fish_length_px = (fish_length * FOCAL_LENGTH / distance_from_camera) * (1.0 / PIXEL_SIZE_M)
    ground_truth_disparity_px = (FOCAL_LENGTH * BASELINE / distance_from_camera) * (PIXEL_COUNT_WIDTH / IMAGE_SENSOR_WIDTH)
    predicted_head_disparity_px = ground_truth_disparity_px + head_disparity_error_px
    predicted_tail_disparity_px = ground_truth_disparity_px + tail_disparity_error_px

    # get coordinates of head and tail in left image (attribute disparity errors evenly to left and right images)
    left_head_coordinates_x_px = PIXEL_COUNT_WIDTH / 2.0 - predicted_head_disparity_px / 2.0 - fish_length_px / 2.0
    right_head_coordinates_x_px = PIXEL_COUNT_WIDTH / 2.0 + predicted_head_disparity_px / 2.0 - fish_length_px / 2.0
    left_tail_coordinates_x_px = PIXEL_COUNT_WIDTH / 2.0 - predicted_tail_disparity_px / 2.0 + fish_length_px / 2.0
    right_tail_coordinates_x_px = PIXEL_COUNT_WIDTH / 2.0 + predicted_tail_disparity_px / 2.0 + fish_length_px / 2.0

    y_px = PIXEL_COUNT_HEIGHT / 2.0
    
    predicted_head_distance_from_camera = (FOCAL_LENGTH * BASELINE / predicted_head_disparity_px) * (PIXEL_COUNT_WIDTH / IMAGE_SENSOR_WIDTH)
    predicted_tail_distance_from_camera = (FOCAL_LENGTH * BASELINE / predicted_tail_disparity_px) * (PIXEL_COUNT_WIDTH / IMAGE_SENSOR_WIDTH)

    head_coordinates_world = convert_to_world_point(left_head_coordinates_x_px, y_px, predicted_head_distance_from_camera)
    tail_coordinates_world = convert_to_world_point(left_tail_coordinates_x_px, y_px, predicted_tail_distance_from_camera)
    distance = np.linalg.norm(head_coordinates_world-tail_coordinates_world)
    
    absolute_error = distance - fish_length
    relative_error = (absolute_error / fish_length) * 100
    return absolute_error, relative_error


<h3> Distance from camera: 1 meter </h3>

In [ ]:
def plot_relative_errors(fish_length, distance_from_camera, max_disparity_error, increment=5):
    disparity_errors = np.arange(10, max_disparity_error, increment)
    all_relative_errors = {}
    for disparity_error in disparity_errors:
        absolute_errors, relative_errors = [], []
        for head_disparity_error_px in np.arange(-disparity_error, disparity_error, increment):
            for tail_disparity_error_px in np.arange(-disparity_error, disparity_error, increment):
                absolute_error, relative_error = get_length_error(fish_length, distance_from_camera, head_disparity_error_px, tail_disparity_error_px)
                absolute_errors.append(absolute_error)
                relative_errors.append(relative_error)
        all_relative_errors[disparity_error] = relative_errors

#     fig, ax = plt.subplots(figsize=(10, 5))
#     b = generate_box_plots(ax, all_relative_errors)
#     plt.xticks(range(1, len(disparity_errors)+1), list(disparity_errors))
#     plt.show()

    data = [all_relative_errors[disparity_error] for disparity_error in disparity_errors]
    plt.figure(figsize=(15, 10))
    plt.boxplot(data)
    plt.xticks(range(1, len(disparity_errors)+1), list(disparity_errors))
    plt.ylim(top=min(100, plt.ylim()[1]))
    plt.xlabel('Maximum disparity error in pixels')
    plt.ylabel('Relative Error Distribution for Head-Tail Distance (pct)')
    plt.title('Fish Length: {}m, Distance from Camera: {}m'.format(fish_length, distance_from_camera))
    plt.grid()
    plt.show()


In [ ]:
plot_relative_errors(0.5, 1.0, 100)

In [ ]:
plot_relative_errors(0.5, 2.0, 60, increment=2)

In [ ]:
plot_relative_errors(0.5, 3.0, 30, 2)

In [ ]:
plot_relative_errors(0.5, 4.0, 30, 2)